In [ ]:
from pyspark.sql.functions import *
import string
import string
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


full_data = spark.read.option("header", "true").option("mode", "DROPMALFORMED").csv("winemag-data-130k-v2.csv")
full_data = full_data.select('points','description').dropna()

In [ ]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet
from pyspark.sql.functions import udf

def splitForWordnet(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    


lemma = WordNetLemmatizer()
analyzer = SentimentIntensityAnalyzer() 


def cleanText(data):
    tokens = word_tokenize(data)
    tokens = [word.lower() for word in tokens]

    punc = str.maketrans('', '', string.punctuation)
    replaced = [word.translate(punc) for word in tokens]
    all_stop_words = set(stopwords.words('english'))
    data = [word for word in replaced if word not in all_stop_words]
    

    data = [t for t in data if len(t) > 0]
    pos_tags = pos_tag(data)
    data = [lemma.lemmatize(t[0], splitForWordnet(t[1])) for t in pos_tags]
    data = [t for t in data if len(t) > 1]


    return data

def vader(data):
    data = " ".join(data)
    score =  analyzer.polarity_scores(data)
    return score['compound']

clean_udf = udf(cleanText())
vader_udf = udf(vader)

data = full_data.withColumn("review", clean_udf(col("description")))
data = data.withColumn("customer_review_score", vader_udf(col("review")))
data = data.withColumn("label", when(col("description")>80, 1.0).otherwise(0.0))
data = data.select(["description","label","customer_review_score"])

positiveScore = data.filter((col("customer_review_score") >= 0.75))
negetiveScore = data.filter((col("customer_review_score") <= -0.75))        
irony = data.filter((col("customer_review_score") <= -0.75) & (col("label") == 1))    




In [ ]:

irony = irony.rdd
irony.collect()

negetiveScore = negetiveScore.rdd
negetiveScore.collect()

positiveScore = positiveScore.rdd
positiveScore.collect()